In [2]:
import pandas as pd
import numpy as np

!pip3 install surprise
from surprise import Reader
from surprise import Dataset

from surprise import SVD, KNNBasic, NMF, SlopeOne, CoClustering #other knn, randoms, not svdpp because were not doing implicit               # importer ici les algo qu'on testera
from surprise import model_selection
from surprise import dump

from sklearn.model_selection import train_test_split 

In [3]:
df = pd.read_csv("Datasets/data_train.csv")

df[["user", "item"]] = df.Id.str.split("_", expand=True)

df.user = df.user.str.replace("r", "")
df.item = df.item.str.replace("c", "")

#########
df2 = pd.read_csv("Datasets/sample_submission.csv")

df2[["user", "item"]] = df2.Id.str.split("_", expand=True)

df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")

df

,Id,Prediction,user,item
0,r44_c1,4,44,1
1,r61_c1,3,61,1
2,r67_c1,4,67,1
3,r72_c1,3,72,1
4,r86_c1,5,86,1
...,...,...,...,...
1176947,r9990_c1000,4,9990,1000
1176948,r9992_c1000,5,9992,1000
1176949,r9994_c1000,3,9994,1000
1176950,r9997_c1000,4,9997,1000


In [4]:
df_fifth, test = train_test_split(df, test_size=0.8, random_state=1)
df_hundredth,test = train_test_split(df, test_size=0.99, random_state=1)

In [5]:
reader = Reader(rating_scale=(1,5)) 
data = Dataset.load_from_df(df[["user","item","Prediction"]], reader)
data_fifth = Dataset.load_from_df(df_fifth[["user","item","Prediction"]], reader)
data_hundredth = Dataset.load_from_df(df_hundredth[["user","item","Prediction"]], reader)


In [6]:
del df#freeing memory
del df_fifth
del df_hundredth

### SVD

In [13]:
param_grid = {
    'n_factors' : [10],
    'n_epochs': [20],
    'lr_all': [0.003],
    'reg_all': [0.05]
} 
cv=5
algorithm = SVD

gs = model_selection.GridSearchCV(algorithm, param_grid, measures=['rmse'], cv=cv, n_jobs=-5, joblib_verbose=10)  #enlever mae car non utilisé dans le projet pour sauver du temps
gs.fit(data_hundredth)
print(gs.best_params)
print(gs.best_score)
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())
dump.dump("dump/SVDfitted_dump", algo=algo, verbose=1)

[Parallel(n_jobs=-5)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-5)]: Done   2 out of   5 | elapsed:    1.1s remaining:    1.7s
[Parallel(n_jobs=-5)]: Done   3 out of   5 | elapsed:    1.4s remaining:    0.9s
[Parallel(n_jobs=-5)]: Done   5 out of   5 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=-5)]: Done   5 out of   5 | elapsed:    1.7s finished


{'rmse': {'n_factors': 10, 'n_epochs': 20, 'lr_all': 0.003, 'reg_all': 0.05}}
{'rmse': 1.061329011710672}
The dump has been saved as file dump/SVDfitted_dump


### KNN Basic

In [14]:
param_grid = {
    'k' : [40,100],
    'min_k': [1,3]
} 
cv=5
algorithm = KNNBasic

gs = model_selection.GridSearchCV(algorithm, param_grid, measures=['rmse'], cv=cv, n_jobs=-5, joblib_verbose=10)  #enlever mae car non utilisé dans le projet pour sauver du temps
gs.fit(data_hundredth)
print(gs.best_params)
print(gs.best_score)
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())
dump.dump("dump/KNN_basic_fitted_dump", algo=algo, verbose=1)

[Parallel(n_jobs=-5)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-5)]: Done   5 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-5)]: Done  10 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-5)]: Done  16 out of  20 | elapsed:   15.9s remaining:    4.0s
[Parallel(n_jobs=-5)]: Done  20 out of  20 | elapsed:   17.0s finished


{'rmse': {'k': 40, 'min_k': 3}}
{'rmse': 1.117062845195538}
Computing the msd similarity matrix...
Done computing similarity matrix.
The dump has been saved as file dump/KNN_basic_fitted_dump


### NMF

In [15]:
param_grid = {
    'n_factors' : [15],
    'n_epochs': [50]
} 
cv=5
algorithm = NMF

gs = model_selection.GridSearchCV(algorithm, param_grid, measures=['rmse'], cv=cv, n_jobs=-5, joblib_verbose=10)  #enlever mae car non utilisé dans le projet pour sauver du temps
gs.fit(data_hundredth)
print(gs.best_params)
print(gs.best_score)
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())
dump.dump("dump/NMFfitted_dump", algo=algo, verbose=1)

[Parallel(n_jobs=-5)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-5)]: Done   2 out of   5 | elapsed:    1.9s remaining:    2.9s
[Parallel(n_jobs=-5)]: Done   3 out of   5 | elapsed:    2.3s remaining:    1.5s
[Parallel(n_jobs=-5)]: Done   5 out of   5 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=-5)]: Done   5 out of   5 | elapsed:    2.7s finished


{'rmse': {'n_factors': 15, 'n_epochs': 50}}
{'rmse': 1.3240168225571949}
The dump has been saved as file dump/NMFfitted_dump


### SlopeOne

In [17]:
algo = SlopeOne()
algo.fit(data.build_full_trainset())
dump.dump("dump/SlopeOne_fitted_dump", algo=algo, verbose=1)

The dump has been saved as file dump/SlopeOne_fitted_dump


### CoClustering

In [18]:
param_grid = {
    'n_cltr_u' : [3],
    'n_cltr_i' : [3], 
    'n_epochs': [50]
} 
cv=5
algorithm = CoClustering

gs = model_selection.GridSearchCV(algorithm, param_grid, measures=['rmse'], cv=cv, n_jobs=-5, joblib_verbose=10)  #enlever mae car non utilisé dans le projet pour sauver du temps
gs.fit(data_hundredth)
print(gs.best_params)
print(gs.best_score)
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())
dump.dump("dump/CoClustering_fitted_dump", algo=algo, verbose=1)

[Parallel(n_jobs=-5)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-5)]: Done   2 out of   5 | elapsed:    3.5s remaining:    5.2s
[Parallel(n_jobs=-5)]: Done   3 out of   5 | elapsed:    3.8s remaining:    2.5s
[Parallel(n_jobs=-5)]: Done   5 out of   5 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=-5)]: Done   5 out of   5 | elapsed:    4.7s finished


{'rmse': {'n_cltr_u': 3, 'n_cltr_i': 3, 'n_epochs': 50}}
{'rmse': 1.2605553912494378}
The dump has been saved as file dump/CoClustering_fitted_dump


## Estimation

In [27]:
del algo, gs

array_SVD = np.ones((df2.shape[0],1))
array_KNN = np.ones((df2.shape[0],1))
array_NMF = np.ones((df2.shape[0],1))
array_SlopeOne = np.ones((df2.shape[0],1))
array_CoClustering = np.ones((df2.shape[0],1))

_, algo_svd = dump.load("dump/SVDfitted_dump")
_, algo_knn = dump.load("dump/KNN_basic_fitted_dump")
_, algo_nmf = dump.load("dump/NMFfitted_dump")
_, algo_slopeone = dump.load("dump/SlopeOne_fitted_dump")
_, algo_coclustering = dump.load("dump/CoClustering_fitted_dump")

In [28]:
knn_fail = 0
slopeone_fail = 0

for i in df2.iterrows():
    array_SVD[i[0]] = algo_svd.estimate(int(i[1][2]), int(i[1][3]))
    
    try:
        array_KNN[i[0]]= algo_knn.estimate(int(i[1][2]), int(i[1][3]))[0]
    except Exception as e: 
        array_KNN[i[0]]=-1000
        knn_fail += 1
        
    array_NMF[i[0]] = algo_nmf.estimate(int(i[1][2])-1, int(i[1][3])-1)
    
    try:
        array_SlopeOne[i[0]]=algo_slopeone.estimate(int(i[1][2]), int(i[1][3]))
    except Exception as e: 
        array_SlopeOne[i[0]]=-1000
        slopeone_fail += 1
    
    array_CoClustering[i[0]] = algo_coclustering.estimate(int(i[1][2]), int(i[1][3]))
    if i[0]%100000==0:
        print(i[0])
    

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000


In [29]:
print(knn_fail)
print(slopeone_fail)

1549
1549
